In [ ]:
from pydub import AudioSegment
import os
import pandas as pd

In [ ]:
def speed_up_wav(input_folder, output_folder, target_duration):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    data = {'File': [], 'Original Duration (ms)': [], 'Playback speed': [], 'Sped-up Duration (ms)': []}

    for filename in os.listdir(input_folder):
        if filename.endswith(".mp3"):
            file_path = os.path.join(input_folder, filename)

            # Load the original audio file
            original_audio = AudioSegment.from_mp3(file_path)

            # Calculate the duration of the original audio in milliseconds
            original_duration = len(original_audio)

            # Check if the original duration is less than or equal to the target duration
            if original_duration <= target_duration:
                print(f"Skipped file {filename}: Original duration is less than or equal to {target_duration} ms.")
                
                # Include skipped file information in the DataFrame
                data['File'].append(filename)
                data['Original Duration (ms)'].append(original_duration)
                data['Playback speed'].append(None)
                data['Sped-up Duration (ms)'].append(None)

                # Create the output file path for skipped files
                output_path = os.path.join(output_folder, f"{filename.replace('.wav', '.wav')}")

                # Export the original audio to the output folder for skipped files
                original_audio.export(output_path, format="wav")

                print(f"Skipped file saved to: {output_path}")
                continue

            if original_duration == 2208:
                speed_factor = 1.01

                # Speed up the audio by the factor
                sped_up_audio = original_audio.speedup(playback_speed=speed_factor)

                # Create the output file path
                output_path = os.path.join(output_folder, f"{filename.replace('.mp3', '.wav')}")

                # Export the modified audio to the output folder
                sped_up_audio.export(output_path, format="wav")

                data['File'].append(filename)
                data['Original Duration (ms)'].append(original_duration)
                data['Playback speed'].append(speed_factor)
                data['Sped-up Duration (ms)'].append(sped_up_duration)

                print(f"\nFinal Result for {filename}:")
                print(f"Calculated Speed Factor: {speed_factor}")
                print(f"Sped-up Duration: {sped_up_duration} ms")
                print(f"Saved to: {output_path}")
                print("-" * 30)

                continue


            # Initialize the speed factor to achieve the target duration
            speed_factor = round(original_duration / target_duration, 2)

            # Speed up the audio by the calculated factor
            sped_up_audio = original_audio.speedup(playback_speed=speed_factor)

            # Calculate the duration of the sped-up audio in milliseconds
            sped_up_duration = len(sped_up_audio)

            while sped_up_duration > target_duration:
                speed_factor += 0.05

                # Speed up the audio by the updated factor
                sped_up_audio = original_audio.speedup(playback_speed=speed_factor)

                # Recalculate the duration of the sped-up audio
                sped_up_duration = len(sped_up_audio)
            
            # Store data in the dictionary
            data['File'].append(filename)
            data['Original Duration (ms)'].append(original_duration)
            data['Playback speed'].append(speed_factor)
            data['Sped-up Duration (ms)'].append(sped_up_duration)

            # Create the output file path
            output_path = os.path.join(output_folder, f"{filename.replace('.mp3', '.wav')}")

            # Export the modified audio to the output folder
            sped_up_audio.export(output_path, format="wav")

            print(f"\nFinal Result for {filename}:")
            print(f"Calculated Speed Factor: {speed_factor}")
            print(f"Sped-up Duration: {sped_up_duration} ms")
            print(f"Saved to: {output_path}")
            print("-" * 30)

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    excel_output_path = os.path.join(output_folder, "audio_processing_results.xlsx")
    df.to_excel(excel_output_path, index=False)
    print(f"\nResults saved to Excel file: {excel_output_path}")

In [ ]:
input_folder = "[folder where your input audios are stored]"
output_folder = "[folder where you want to store your sped up audios]"
# set limit of duration of audios in ms (can last max n seconds)
target_duration = 2200

speed_up_wav(input_folder, output_folder, target_duration)